### Quiero poder predecir mediante un modelo de Machine Learning la puntuación de una película. Me quiero basar en las puntuaciones de películas de la web Filmaffinity. Por lo que recopilaré buna parte de las películas de la web, teniendo en cuenta que tengan más de 500 votos, no sean telefilmes ni películas de animación.

Importo las librerías necesarias

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import requests
import time

Conecto mediante Selenium con la URL principal (listado de todas las películas) donde voy a conseguir las URLs de la ficha de cada película y obtener los datos como: nombre, año, director, actores, puntuación, etc...

In [21]:

# Conectamos con la web y abrimos ventana navegador en blanco
service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# Conectamos con la web de Filmaffinity
driver.get("https://www.filmaffinity.com/es/topgen.php?genres=&chv=0&movietype=movie%7Cex-anim%3Aex-tv&country=&fromyear=1874&toyear=2023&ratingcount=2&runtimemin=0&runtimemax=4")

# Clickamos el botón de aceptar cookies
Cookies = driver.find_element(by="xpath", value="/html/body/div[1]/div/div/div/div[2]/div/button[2]/span")
Cookies.click()


Creo un buccle para caragar todos los registros completos y poder acceder a las URLs de las fichas de las películas

In [3]:
# Creamos un bucle para hacer scroll y cargar los resultados de todas las peliculas

for _ in range(750):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)
    Bajar = driver.find_element(by = By.XPATH, value="/html/body/div[5]/table/tbody/tr/td[2]/div[1]/div[4]/div[5]/div/i")
    Bajar.click()

Mediante Beautiful Soup accedo al contenido de la web y extraigo (posición de la película en la lista, nombre y URL de la ficha de la película)

In [23]:
soup = bs(driver.page_source,"lxml")
info_peliculas = soup.select("div.mc-info-container")

In [38]:
posiciones = soup.select("li.position")
enlace_pelicula = soup.select("div.mc-title")[0].find("a")["href"]
nombre_pelicula = soup.select("div.mc-title")[0].find("a")["title"].strip()

In [42]:
posicion_pelicula = [posicion.get_text() for posicion in posiciones]
enlaces_peliculas = [enlace.select("div.mc-title")[0].find("a")["href"] for enlace in info_peliculas]
nombres_peliculas = [nombre.select("div.mc-title")[0].find("a")["title"].strip() for nombre in info_peliculas]

Logro una listado de 15850 URLs de películas

In [43]:
len(nombres_peliculas)

15850

Convierto el diccionario donde tengo los datos en un dataframe

In [44]:
columnas = ["posicion", "pelicula", "enlace"]
datos = [posicion_pelicula, nombres_peliculas, enlaces_peliculas]

peliculas_enlaces_15850 = {col:dat for col, dat in zip(columnas, datos)}
df_pelis_enlaces_15850 = pd.DataFrame(peliculas_enlaces_15850)

Exporto dicho dataframe a un fichero CSV

In [3]:
# df_pelis_enlaces_15850.to_csv("..\data\raw\15850_mejores_peliculas_FilmAffinity_enlaces.csv", encoding= "utf16", index=False)

Leo ese CSV para poder accder a cada URL y extraer la información de cada película

In [5]:
df_pelis_enlaces = pd.read_csv(r"..\data\raw\15850_mejores_peliculas_FilmAffinity_enlaces.csv", encoding= "utf16", index_col=0)

Porciono el dataset en subsets para ir extrayendo la información más facilmente y no tardar tanto en cada proceso de extracción de información

In [10]:
df_pelis_enlaces_001 = df_pelis_enlaces.iloc[:1585,:]
df_pelis_enlaces_002 = df_pelis_enlaces.iloc[1585:3170,:]
df_pelis_enlaces_003 = df_pelis_enlaces.iloc[3170:4755,:]
df_pelis_enlaces_004 = df_pelis_enlaces.iloc[4755:6340,:]
df_pelis_enlaces_005 = df_pelis_enlaces.iloc[6340:9510,:]
df_pelis_enlaces_006 = df_pelis_enlaces.iloc[9510:11095,:]
df_pelis_enlaces_007 = df_pelis_enlaces.iloc[11095:12680,:]
df_pelis_enlaces_008 = df_pelis_enlaces.iloc[12680:14265,:]
df_pelis_enlaces_009 = df_pelis_enlaces.iloc[14265:,:]

In [11]:
response = requests.get("https://www.filmaffinity.com/es/main.html")
session = requests.Session()
session.close() 

Creo un diccionario donde iré reuniendo la información que vaya extrayendo de la web

In [4]:
dicc_peliculas = {
                    "titulo_castellano": [],
                    "titulo_original": [],
                    "year": [],
                    "duracion": [],
                    "pais": [],
                    "director": [],
                    "guion": [],
                    "musica": [],
                    "fotografia": [],
                    "productora" : [],
                    "genero": [],
                    "actores": [],
                    "sinopsis": [],
                    "tags": [],
                    "votos": [],
                    "reviews":[],
                    "puntuacion" : []
                   }

Creo una función que se encargará de automatizar el proceso de recopilación de datos

In [5]:
def get_data(dataframe):
        
        for url in dataframe["enlace"]:
            
            try:
                 
                response = requests.get(url)
                session = requests.Session()
                html = response.content
                sopa = bs(html, "lxml")

                dicc_peliculas["titulo_castellano"].append(sopa.find_all("span", {"itemprop": "name"})[0].get_text().strip())
                dicc_peliculas["titulo_original"].append(sopa.find_all("dl", {"class": "movie-info"})[0].find("dd").get_text().replace("\n","").strip())
                dicc_peliculas["year"].append(sopa.find_all("dd", {"itemprop": "datePublished"})[0].get_text())
                dicc_peliculas["duracion"].append(sopa.find_all("dd", {"itemprop": "duration"})[0].get_text().split()[0])
                dicc_peliculas["pais"].append(sopa.find_all("span", {"id": "country-img"})[0].find("img")["alt"])
                dicc_peliculas["director"].append(sopa.find_all("span", {"itemprop": "name"})[1].get_text())
                dicc_peliculas["guion"].append(sopa.find_all("div", {"class": "credits"})[1].find_all("span",{"class":"nb"})[0].find("span").get_text())
                dicc_peliculas["musica"].append(sopa.find_all("div", {"class": "credits"})[2].find("span").find("span").get_text())

                dicc_peliculas["fotografia"].append(sopa.find_all("div", {"class": "credits"})[3].find("span").find("span").get_text())
                dicc_peliculas["productora"].append(soup.find_all("dd", {"class":"card-producer"})[0].find("span").find("span").get_text().strip())             
                dicc_peliculas["genero"].append(sopa.find_all("span", {"itemprop": "genre"})[0].get_text())

                reparto = sopa.find_all("a", {"itemprop": "url"})[1:6]
                actores = []
                for actor in reparto:
                    actores.append(actor["title"])
                dicc_peliculas["actores"].append(actores)

                dicc_peliculas["sinopsis"].append(sopa.find('dd', itemprop='description').get_text())

                etiquetas = sopa.find_all("dd", {"class": "card-genres"})[0].find_all("a")
                tags = []
                for etiqueta in etiquetas:
                    tags.append(etiqueta.get_text())
                dicc_peliculas["tags"].append(tags)
                
                dicc_peliculas["votos"].append(sopa.find_all("div", {"id": "movie-count-rat"})[0].get_text().replace("\n","").replace(".","").split("votos")[0])
                dicc_peliculas["reviews"].append(sopa.find_all("div", {"id": "movie-reviews-box"})[0].get_text().replace("\n","").replace(" ","").split("críticas")[0])
                dicc_peliculas["puntuacion"].append(sopa.find_all("div", {"id": "movie-rat-avg"})[0].get_text().strip())
                
            except:
                pass
        session.close()        
        return dicc_peliculas

In [12]:
len(df_pelis_enlaces_005)

3170

In [13]:
get_data(df_pelis_enlaces_005)

Me aseguro que cada clave del diccionario tienen la misma longitud para así poder crear el dataframe con todos los registros

In [14]:
for key, value in dicc_peliculas.items():
    print(key, len(value))

Exporto los registros obtenidos a un fichero CSV para poder posteriormente continuar con el proceso de limpieza y análisis de los datos

In [74]:
# df_peliculas_005 = pd.DataFrame(dicc_peliculas)
# df_peliculas_005.to_csv("..\data\raw\fichas_peliculas\pelis_005.csv", encoding= "utf16", index= False)

### Para que el algoritmo tenga mayor información para predecir quiero integrar en los datos los principales premios que pueden recibir las películas.

Primero obtengo los enlaces de todos los festivales de cine y premios de academias de cine de todos los paises

In [7]:
response = requests.get("https://www.filmaffinity.com/es/all_awards.php")
html = response.content
sopa = bs(html, "lxml")

enlaces = sopa.find_all("div",{"class":"award-container"})
enlaces_festivales_lista = []
for award in enlaces:
    a_tag = award.find("a")
    if a_tag is not None:
        enlaces_festivales_lista.append(award.find("a")["href"])
    else:
        enlaces_festivales_lista.append("Link not found")

Obtengo de cada URL de cada festival de cine las URL de las 2 categorías principales, que corresponden a películas

In [9]:
categorias_festivales_lista = []
for url in enlaces_festivales_lista:

  url_001 = url.split("=")[1]
  new_url = url[:37]+"-edition.php?edition-id="+url_001+"_2022"

  response = requests.get(new_url)
  html = response.content
  sopa = bs(html, "lxml")
  
  categorias_awards = sopa.find_all("div", {"class":"vwacat"})
  
  for categoria in categorias_awards:
    categorias_festivales_lista.append(categoria.find("a")["href"])

Guardo los resultados en un dataframe

In [4]:
df_festivales_awards = pd.DataFrame(categorias_festivales_lista)

In [5]:
# df_festivales_awards.to_csv(r"..\data\raw\enlaces_todas_categorias_todos_festivales.csv", encoding="utf16", index = False)

Guardo el dataframe en un csv para poder importarlo después

In [10]:
df_festivales_awards = pd.read_csvr(r"..\data\raw\enlaces_todas_categorias_todos_festivales.csv", encoding= "utf16", index_col=0)

In [12]:
list_definitive = list(df_festivales_awards["0"])

Obtengo una lista de los enlaces de todas las categorías de cada festival para acceder al histórico de películas de todos las años

Premios Razzie

In [14]:
peliculas_festivales_lista = []
dicc_peliculas_certamenes = {}
categorias_festivales_lista =[]

response = requests.get("https://www.filmaffinity.com/es/award-edition.php?edition-id=razzie_2022")
html = response.content
sopa = bs(html, "lxml")

categorias_awards = sopa.find_all("div", {"class":"vwacat"})

for categoria in categorias_awards:
    categorias_festivales_lista.append(categoria.find("a")["href"])


In [15]:
for url in categorias_festivales_lista:
    pelis_por_premio = []
    nombre_festivales_lista = []    
    categoria_festivales_lista = []

    response = requests.get(url)
    html = response.content
    sopa = bs(html, "lxml")

    datos_pelicula = sopa.find_all("span", {"class":"movie-title-link"})
    datos_director = sopa.find_all("span", {"class":"nb"})
    datos_year = sopa.find_all("div", {"class":"year"})

    festival = sopa.find_all("h1")[0].get_text().split("-",1)[0].strip()
    categoria = sopa.find_all("h1")[0].get_text().split("-",1)[1].split("Todos los ganadores")[0].strip()

    for pelicula, director, year  in zip(datos_pelicula, datos_director, datos_year):
        pelicula = pelicula.find("a").get_text().strip()
        director = director.find("a").get_text().replace('"',"").strip()
        years = year.find("a").get_text().strip()
        peliculas_festivales_lista.append(f"{festival} @ {categoria} @ {years} @ {pelicula} @ {director}")

In [16]:
dicc_peliculas_certamenes["premios_peliculas"] = peliculas_festivales_lista
df_premios_peliculas = pd.DataFrame(dicc_peliculas_certamenes)
# df_premios_peliculas.to_csv(r"..\data\raw\peliculas_razzie.csv", encoding="utf16")

In [17]:
reparto_festivales_lista = []
dicc_reparto_certamenes = {}

for url in categorias_festivales_lista:

    pelis_por_premio = []
    nombre_festivales_lista = []    
    categoria_festivales_lista = []

    response = requests.get(url)
    html = response.content
    sopa = bs(html, "lxml")

    datos_reparto = sopa.find_all("div", {"class":"nom-text"})
    datos_pelicula = sopa.find_all("span", {"class":"movie-title-link"})
    datos_director = sopa.find_all("div", {"class":"credits"})
    datos_year = sopa.find_all("div", {"class":"year"})

    festival = sopa.find_all("h1")[0].get_text().split("-",1)[0].strip()
    categoria = sopa.find_all("h1")[0].get_text().split("-",1)[1].split("Todos los ganadores")[0].strip()
    
    for reparto, pelicula, director, year  in zip(datos_reparto, datos_pelicula, datos_director, datos_year):
        reparto = reparto.get_text().strip()
        pelicula = pelicula.find("a").get_text().strip()
        director = director.find("a").get_text().replace('"',"").strip()
        years = year.find("a").get_text().strip()
        reparto_festivales_lista.append(f"{festival} @ {categoria} @ {years} @ {reparto} @ {pelicula} @ {director}")

In [18]:
dicc_reparto_certamenes["premios_reparto"] = reparto_festivales_lista
df_premios_reparto = pd.DataFrame(dicc_reparto_certamenes)
# df_premios_reparto.to_csv("..\data\raw\reparto_razzie.csv", encoding="utf16")

In [15]:
list_definitive

['https://www.filmaffinity.com/es/awards-history.php?cat-id=cannes_golden_palm',
 'https://www.filmaffinity.com/es/awards-history.php?cat-id=cannes_grand_prix',
 'https://www.filmaffinity.com/es/awards-history.php?cat-id=cannes_best_director',
 'https://www.filmaffinity.com/es/awards-history.php?cat-id=cannes_best_actor',
 'https://www.filmaffinity.com/es/awards-history.php?cat-id=cannes_best_actress',
 'https://www.filmaffinity.com/es/awards-history.php?cat-id=cannes_best_screenplay',
 'https://www.filmaffinity.com/es/awards-history.php?cat-id=cannes_fipresci',
 'https://www.filmaffinity.com/es/awards-history.php?cat-id=cannes_camera_d_or',
 'https://www.filmaffinity.com/es/awards-history.php?cat-id=cannes_jury_prize',
 'https://www.filmaffinity.com/es/awards-history.php?cat-id=cannes_best_short_film',
 'https://www.filmaffinity.com/es/awards-history.php?cat-id=cannes_un_certain_regard_best_film',
 'https://www.filmaffinity.com/es/awards-history.php?cat-id=cannes_un_certain_regard_jur

A partir de cada URL obtengo informacion del historio de peliculas (sólo peliculas) premiadas de todos los festivales

In [15]:
peliculas_festivales_lista = []
dicc_peliculas_certamenes = {}

for url in list_definitive:

    pelis_por_premio = []
    nombre_festivales_lista = []    
    categoria_festivales_lista = []

    response = requests.get(url)
    html = response.content
    sopa = bs(html, "lxml")

    datos_pelicula = sopa.find_all("span", {"class":"movie-title-link"})
    datos_director = sopa.find_all("span", {"class":"nb"})
    datos_year = sopa.find_all("div", {"class":"year"})

    festival = sopa.find_all("h1")[0].get_text().split("-",1)[0].strip()
    categoria = sopa.find_all("h1")[0].get_text().split("-",1)[1].split("Todos los ganadores")[0].strip()
    
    for pelicula, director, year  in zip(datos_pelicula, datos_director, datos_year):
        pelicula = pelicula.find("a").get_text().strip()
        director = director.find("a").get_text().replace('"',"").strip()
        years = year.find("a").get_text().strip()
        peliculas_festivales_lista.append(f"{festival} @ {categoria} @ {years} @ {pelicula} @ {director}")

In [14]:
dicc_peliculas_certamenes["premios_peliculas"] = peliculas_festivales_lista
df_premios_peliculas = pd.DataFrame(dicc_peliculas_certamenes)
# df_premios_peliculas.to_csv("..\data\processed\premios_peliculas.csv", encoding="utf16")

A partir de cada UL obtengo informacion del historio de peliculas (directores, actores) premiadas de todos los festivales

In [16]:
reparto_festivales_lista = []
dicc_reparto_certamenes = {}

for url in list_definitive:

    pelis_por_premio = []
    nombre_festivales_lista = []    
    categoria_festivales_lista = []

    response = requests.get(url)
    html = response.content
    sopa = bs(html, "lxml")

    datos_reparto = sopa.find_all("div", {"class":"nom-text"})
    datos_pelicula = sopa.find_all("span", {"class":"movie-title-link"})
    datos_director = sopa.find_all("div", {"class":"credits"})
    datos_year = sopa.find_all("div", {"class":"year"})

    festival = sopa.find_all("h1")[0].get_text().split("-",1)[0].strip()
    categoria = sopa.find_all("h1")[0].get_text().split("-",1)[1].split("Todos los ganadores")[0].strip()
    
    for reparto, pelicula, director, year  in zip(datos_reparto, datos_pelicula, datos_director, datos_year):
        reparto = reparto.get_text().strip()
        pelicula = pelicula.find("a").get_text().strip()
        director = director.find("a").get_text().replace('"',"").strip()
        years = year.find("a").get_text().strip()
        reparto_festivales_lista.append(f"{festival} @ {categoria} @ {years} @ {reparto} @ {pelicula} @ {director}") 

Recopilo los datos, los almaceno en un diccionario y convierto en dataframe y exporto a csv para poder tener tratar la información después

In [9]:
dicc_reparto_certamenes["premios_reparto"] = reparto_festivales_lista
df_premios_reparto = pd.DataFrame(dicc_reparto_certamenes)
# df_premios_reparto.to_csv("..\data\raw\repartos_premiados_todos.csv", encoding="utf16")

In [13]:
len(list_definitive)

865

Extraigo el recuento de todas las etiquetas que existen en la web para poder utilizazr esa información en el análisis e información que proporcionaré al algoritmo

In [2]:
response = requests.get("https://www.filmaffinity.com/es/topics.php")
html = response.content
soup = bs(html, "lxml")

In [3]:
tags = soup.find_all("a", {"class":"topic"})

tag_lista = []
num_tag = []
dicc_tags = {}

for tag in tags:
    tag_lista.append(tag.get_text().split("(")[0].strip())
    num_tag.append(tag.get_text().split("(")[1].replace(")","").strip())

In [4]:
len(tag_lista), len(num_tag)

(415, 415)

In [10]:
import requests

datospelis = []

for i in range(75000,100000):

    url = f"https://api.themoviedb.org/3/movie/{i}?api_key=7b34ecc56572df48f2ce676be26adbeb"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        datospelis.append({
                        'Titulo': data['title'],
                        'Presupuesto': data['budget']
                        })

    # else:
    #     print("Error al obtener la información de la película con el ID:", i)    

In [11]:
# import pandas as pd

# df_pelis_budget = pd.DataFrame(datospelis)
# df_pelis_budget.to_csv("..\data\fichas_peliculas\budget_004.csv", encoding="utf16")